In [ ]:
import pyaudio
import os
import numpy as np
import time

# constants
CHUNK = 1024             # samples per frame
FORMAT = pyaudio.paInt16    # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [ ]:
import sys
import serial

serialPort = '/dev/cu.usbmodem14301'
baudRate = 115200
usbConnection = serial.Serial(serialPort, baudRate)

def commandArduino(string):
    outputStr = string + '\n'
    outputStr = bytes(outputStr, 'utf-8')
    usbConnection.write(outputStr)
    usbConnection.reset_input_buffer()

# Figure out how to auto-scale sensitivity
    - Use a running average
    - Take into account how much is being clipped at the upper range and feed it back into the delta modifier
    
## Or maybe leave sensitivity constant with manual adjustment and allow the user to naturally ignore or pay attention to vibration
## But should avoid clipping upper range, so that loudest sounds always stand out

In [ ]:
class runningAverage:
    def __init__(self, limit):
        self.arr = np.array([] * limit)
        self.limit = limit
    def append(self, value):
        if len(self.arr) >= self.limit:
            self.arr = self.arr[1:]
        self.arr = np.append(self.arr, value)
    def average(self):
        return np.average(self.arr)

In [ ]:
# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# for measuring frame rate
frame_count = 0
frame_limit = 100
start_time = time.time()

mod_value = 0.7
mod_average = runningAverage(100)

while True:
    data = stream.read(CHUNK, exception_on_overflow = False)
    data_np = np.frombuffer(data, dtype=np.int16)
    
    # volume
    delta = np.max(data_np) - np.min(data_np)
    delta_mod = int(np.abs(delta) ** mod_value)
    delta_bounded = min(max(delta_mod, 0), 128)
    
    mod_bounded_diff = delta_mod - delta_bounded
    mod_average.append(mod_bounded_diff ** 0.25)
    mod_value = (1 / (mod_average.average() + 4)) + 0.5

    commandArduino(f'vibrate,{delta_bounded}')
    print(delta, delta_bounded, mod_value)
    
    frame_count += 1

end_time = time.time()
print(frame_limit / (end_time - start_time))